In [17]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib as mpl
import scipy.stats as stats 
import random
import os
from imblearn.over_sampling import SMOTE

# 시각화 옵션 
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('retina')
mpl.rc('font',family='Malgun Gothic')

/var/folders/tt/fc88q57d5pn02r2y5z74ydj80000gn/T/ipykernel_3481/860379959.py:14: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

class CFG:
    SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [19]:
X_train =  pd.get_dummies(train_x.drop(columns=['father','mother','gender']))
test_x = pd.get_dummies(test_x.drop(columns=['father','mother','gender']))
Y_train = train_y


sampler = SMOTE()
sampler.fit(X_train, Y_train)

X_train, Y_train = sampler.fit_resample(X_train, Y_train)

In [20]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [21]:
pipe_list = [('scaler',MinMaxScaler()),
             ('model',SVC())]
pip_model = Pipeline(pipe_list)

In [22]:
hyper_parameter = {'model__C':[1,10,50,100],
                  'model__kernel':['linear'],
                  'model__class_weight':[None, 'balanced']}

grid_model = GridSearchCV(pip_model, 
                         param_grid=hyper_parameter, 
                         cv=5, n_jobs = -1, scoring='f1')

grid_model.fit(X_train, Y_train)

/Users/yungchannelforyou/opt/anaconda3/envs/my_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yungchannelforyou/opt/anaconda3/envs/my_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/yungchannelforyou/opt/anaconda3/envs/my_env/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/Users/yungchannelforyou/opt/anaconda3/envs/my_env/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/yungchannelforyou/opt/anaconda3/envs/my_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1132, in f1_score
    return

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model', SVC())]),
             n_jobs=-1,
             param_grid={'model__C': [1, 10, 50, 100],
                         'model__class_weight': [None, 'balanced'],
                         'model__kernel': ['linear']},
             scoring='f1')

In [23]:
best_model = grid_model.best_estimator_
best_model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model', SVC(C=1, kernel='linear'))])

In [24]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(test_x)
submit = pd.read_csv('sample_submission.csv')
submit['class'] = Y_test_pred

submit.to_csv('submitUpsamleSVM.csv', index=False)

In [27]:
test_y_pred = pd.DataFrame(Y_test_pred)
test_y_pred= test_y_pred.replace({0:'A',1:'B',2:'C'})
test_y_pred.value_counts()

B    87
A    51
C    37
dtype: int64